## Limpeza de dados Cartola ano 2017

In [1]:
from pyspark.sql import HiveContext
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.functions import lower, col, lit, regexp_replace, trim, substring, when, expr, udf, count, sum, monotonically_increasing_id
import pandas as pd
import json
import requests

# Confirguração para não sobrescrever DF
spark.conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

## Funções Custonmizadas

In [2]:
def remove_after_hiphen(team_name):
    if team_name.startswith('atletico') or team_name.startswith('atl')  or team_name.startswith('Atl') or team_name.startswith('Ath'):
        return team_name
    else:
        return team_name.split('-', 1)[0]

In [3]:
#Remove dos valores das Strings o que estiver após os hiphen
remove_hiphen_udf = udf(remove_after_hiphen, StringType())
#partidas_2014_ct = partidas_2014_df.withColumn('away_team', remove_hiphen_udf(partidas_2014_df['away_team']))

## Jogadores

In [6]:
#Carrega arquivo CSV
jogadores_2017_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2017/2017_jogadores.csv", header=True)
sorted_jogadores_2017_df = jogadores_2017_df.sort(jogadores_2017_df.AtletaID.asc())


In [7]:
#Adiciona coluna ANO = 2017
jogadores_2017_ano = sorted_jogadores_2017_df.withColumn('year', lit(2017))
jogadores_2017_ano.toPandas()

,AtletaID,Apelido,ClubeID,PosicaoID,year
0,100065,Lincoln,262,5,2017
1,100132,Lucas Poletto,284,5,2017
2,100648,John Kleber,303,5,2017
3,100651,Rodrygo,277,5,2017
4,100652,Yuri Alberto,277,5,2017
...,...,...,...,...,...
746,99900,Luan Pereira,314,4,2017
747,99903,Lucas Campos,263,5,2017
748,99915,Ezequiel,263,4,2017
749,99918,Patrick,284,4,2017


In [8]:
jogadores_2017_ano = jogadores_2017_ano.withColumnRenamed('AtletaID', 'ID')

In [9]:
jogadores_2017_ano.write.mode('overwrite').partitionBy('year').parquet('/cartola/clean/jogadores/')

In [10]:
#transforma arquivo em parquet
jogadores_2017_parquet = spark.read.option('basePath', '/cartola/clean/jogadores/').parquet('/cartola/clean/jogadores/*')

In [12]:
jogadores_2017_parquet.toPandas()

,ID,Apelido,ClubeID,PosicaoID,year
0,54797,Túlio De Melo,NA,5,2015
1,55519,Edmílson,315,5,2015
2,56102,João Paulo,264,4,2015
3,60752,Cristóvão Borges,NA,6,2015
4,60780,Vinícius Eutrópio,315,6,2015
...,...,...,...,...,...
3806,94067,Matheus Fernandes,263,4,2017
3807,94068,Ribamar,293,5,2017
3808,42222,Osvaldo,292,5,2017
3809,42232,Amaral,315,4,2017


In [13]:
print(jogadores_2017_parquet.count())

3811


## Partidas_2017

In [14]:
#Carrega arquivo CSV
partidas_2017_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2017/2017_partidas.csv", header=True)

In [15]:
partidas_2017_df.show(5)

+---+----+-----+------------------+----------------+-----+------------------+--------------------+----+
|_c0|game|round|              date|       home_team|score|         away_team|               arena|   X|
+---+----+-----+------------------+----------------+-----+------------------+--------------------+----+
|  1|   1|    1|14/05/2017 - 11:00| Fluminense - RJ|3 x 2|       Santos - SP|Maracanã - Rio de...|null|
|  2|   2|    1|13/05/2017 - 16:00|   Flamengo - RJ|1 x 1|     Atlético - MG|Maracanã - Rio de...|null|
|  3|   3|    1|14/05/2017 - 16:00|  Palmeiras - SP|4 x 0|Vasco da Gama - RJ|Allianz Parque - ...|null|
|  4|   4|    1|13/05/2017 - 19:00|Corinthians - SP|1 x 1|  Chapecoense - SC|Arena Corinthians...|null|
|  5|   5|    1|14/05/2017 - 16:00|   Cruzeiro - MG|1 x 0|    São Paulo - SP|Mineirão - Belo H...|null|
+---+----+-----+------------------+----------------+-----+------------------+--------------------+----+
only showing top 5 rows



In [16]:
# Criar nome do time com a string antes do Hífen
partidas_2017_ct = partidas_2017_df.withColumn('away_team', remove_hiphen_udf(partidas_2017_df['away_team']))

partidas_2017_ct = partidas_2017_ct.withColumn('away_team', lower(col('away_team')))

partidas_2017_ct = partidas_2017_ct.withColumn('away_team', trim(col('away_team')))

partidas_2017_ct = partidas_2017_ct.withColumn('home_team', remove_hiphen_udf(partidas_2017_ct['home_team']))

partidas_2017_ct = partidas_2017_ct.withColumn('home_team', lower(col('home_team')))

partidas_2017_ct = partidas_2017_ct.withColumn('home_team', trim(col('home_team')))

partidas_2017_ct = partidas_2017_ct.withColumn('home_score', substring('score', 1,1))\

partidas_2017_ct = partidas_2017_ct.withColumn('away_score', substring('score', 5, 5))\

partidas_2017_ct = partidas_2017_ct.withColumn('home_score', partidas_2017_ct['home_score'].cast(IntegerType()))

partidas_2017_ct = partidas_2017_ct.withColumn('away_score', partidas_2017_ct['away_score'].cast(IntegerType()))

partidas_2017_ct = partidas_2017_ct.withColumn('total_gols', partidas_2017_ct['away_score'] + partidas_2017_ct['home_score'] )

partidas_2017_ct = partidas_2017_ct.withColumn('year', lit(2017))

time_ganhador = expr(
    """IF(home_score > away_score, home_team, IF(home_score = away_score, 'empate', away_team))"""
)

partidas_2017_ct = partidas_2017_ct.withColumn('result', time_ganhador)

partidas_2017_ct.toPandas()

,_c0,game,round,date,home_team,score,away_team,arena,X,home_score,away_score,total_gols,year,result
0,1,1,1,14/05/2017 - 11:00,fluminense,3 x 2,santos,Maracanã - Rio de Janeiro - RJ,None,3,2,5,2017,fluminense
1,2,2,1,13/05/2017 - 16:00,flamengo,1 x 1,atlético - mg,Maracanã - Rio de Janeiro - RJ,None,1,1,2,2017,empate
2,3,3,1,14/05/2017 - 16:00,palmeiras,4 x 0,vasco da gama,Allianz Parque - Sao Paulo - SP,None,4,0,4,2017,palmeiras
3,4,4,1,13/05/2017 - 19:00,corinthians,1 x 1,chapecoense,Arena Corinthians - Sao Paulo - SP,None,1,1,2,2017,empate
4,5,5,1,14/05/2017 - 16:00,cruzeiro,1 x 0,são paulo,Mineirão - Belo Horizonte - MG,None,1,0,1,2017,cruzeiro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,376,376,38,03/12/2017 - 17:00,atlético - pr,3 x 0,palmeiras,Arena da Baixada - Curitiba - PR,None,3,0,3,2017,atlético - pr
376,377,377,38,03/12/2017 - 17:00,sport,1 x 0,corinthians,Ilha do Retiro - Recife - PE,None,1,0,1,2017,sport
377,378,378,38,03/12/2017 - 17:00,vitória,1 x 2,flamengo,Manoel Barradas - Salvador - BA,None,1,2,3,2017,flamengo
378,379,379,38,03/12/2017 - 17:00,atlético - go,1 x 1,fluminense,Olímpico Pedro Ludovico - Goiania - GO,None,1,1,2,2017,empate


In [17]:
# Removendo colunas
partidas_2017_ct = partidas_2017_ct.drop('_c0', 'X', 'game_id')
# Criando ID para DF
# partidas_2017_ct = partidas_2017_ct.withColumn('game_id', monotonically_increasing_id())
partidas_2017_ct.toPandas()

,game,round,date,home_team,score,away_team,arena,home_score,away_score,total_gols,year,result
0,1,1,14/05/2017 - 11:00,fluminense,3 x 2,santos,Maracanã - Rio de Janeiro - RJ,3,2,5,2017,fluminense
1,2,1,13/05/2017 - 16:00,flamengo,1 x 1,atlético - mg,Maracanã - Rio de Janeiro - RJ,1,1,2,2017,empate
2,3,1,14/05/2017 - 16:00,palmeiras,4 x 0,vasco da gama,Allianz Parque - Sao Paulo - SP,4,0,4,2017,palmeiras
3,4,1,13/05/2017 - 19:00,corinthians,1 x 1,chapecoense,Arena Corinthians - Sao Paulo - SP,1,1,2,2017,empate
4,5,1,14/05/2017 - 16:00,cruzeiro,1 x 0,são paulo,Mineirão - Belo Horizonte - MG,1,0,1,2017,cruzeiro
...,...,...,...,...,...,...,...,...,...,...,...,...
375,376,38,03/12/2017 - 17:00,atlético - pr,3 x 0,palmeiras,Arena da Baixada - Curitiba - PR,3,0,3,2017,atlético - pr
376,377,38,03/12/2017 - 17:00,sport,1 x 0,corinthians,Ilha do Retiro - Recife - PE,1,0,1,2017,sport
377,378,38,03/12/2017 - 17:00,vitória,1 x 2,flamengo,Manoel Barradas - Salvador - BA,1,2,3,2017,flamengo
378,379,38,03/12/2017 - 17:00,atlético - go,1 x 1,fluminense,Olímpico Pedro Ludovico - Goiania - GO,1,1,2,2017,empate


In [18]:
partidas_2017_ct.write.mode('overwrite').partitionBy('year').parquet('/cartola/clean/partidas/')

In [20]:
#transforma arquivo em parquet
partidas_2017_parquet = spark.read.option('basePath', '/cartola/clean/partidas/').parquet('/cartola/clean/partidas/*')

In [21]:
partidas_2017_parquet.toPandas()


,game,round,date,home_team,score,away_team,arena,home_score,away_score,total_gols,result,year
0,1,1,20/04/2014 - 18:30,flamengo,0 x 0,goiás,Mané Garrincha - Brasilia - DF,0.0,0.0,0.0,empate,2014
1,2,1,19/04/2014 - 18:30,fluminense,3 x 0,figueirense,Maracanã - Rio de Janeiro - RJ,3.0,0.0,3.0,fluminense,2014
2,3,1,20/04/2014 - 16:00,são paulo,3 x 0,botafogo,Morumbi - Sao Paulo - SP,3.0,0.0,3.0,são paulo,2014
3,4,1,20/04/2014 - 18:30,santos,1 x 1,sport,Vila Belmiro - Santos - SP,1.0,1.0,2.0,empate,2014
4,5,1,20/04/2014 - 16:00,atletico - pr,1 x 0,grêmio,Orlando Scarpelli - Florianopolis - SC,1.0,0.0,1.0,atletico - pr,2014
...,...,...,...,...,...,...,...,...,...,...,...,...
1515,376,38,06/12/2015 - 17:00,atlético - mg,3 x 0,chapecoense,Mineirão - Belo Horizonte - MG,3.0,0.0,3.0,atlético - mg,2015
1516,377,38,06/12/2015 - 17:00,figueirense,1 x 0,fluminense,Orlando Scarpelli - Florianopolis - SC,1.0,0.0,1.0,figueirense,2015
1517,378,38,06/12/2015 - 17:00,coritiba,0 x 0,vasco da gama,Couto Pereira - Curitiba - PR,0.0,0.0,0.0,empate,2015
1518,379,38,06/12/2015 - 17:00,goiás,0 x 1,são paulo,Serra Dourada - Goiania - GO,0.0,1.0,1.0,são paulo,2015


In [23]:
partidas_2017_parquet.schema

StructType(List(StructField(game,StringType,true),StructField(round,StringType,true),StructField(date,StringType,true),StructField(home_team,StringType,true),StructField(score,StringType,true),StructField(away_team,StringType,true),StructField(arena,StringType,true),StructField(home_score,IntegerType,true),StructField(away_score,IntegerType,true),StructField(total_gols,IntegerType,true),StructField(result,StringType,true),StructField(year,IntegerType,true)))

## Times

In [26]:
times_2017_df = spark.read.option('delimiter', ';').option("encoding", "UTF-8").csv("/cartola/data/2017/2017_times.csv", header=True)

sorted_times_2017_df = times_2017_df.sort(times_2017_df.ID.asc())

#Chegar se o arquivo está no hdfs

times_2017_ano = times_2017_df.withColumn('year', lit(2017))
times_2017_ano.toPandas()

,ID,Nome,Abreviacao,Slug,year
0,373,Atlético-GO,ATL,Atlético - GO,2017
1,282,Atlético-MG,ATL,Atlético - MG,2017
2,293,Atlético-PR,ATL,Atlético - PR,2017
3,314,Avaí,AVA,Avaí - SC,2017
4,265,Bahia,BAH,Bahia - BA,2017
5,263,Botafogo,BOT,Botafogo - RJ,2017
6,315,Chapecoense,CHA,Chapecoense - SC,2017
7,264,Corinthians,COR,Corinthians - SP,2017
8,294,Coritiba,COR,Coritiba - PR,2017
9,283,Cruzeiro,CRU,Cruzeiro - MG,2017


In [27]:
times_2017_ano.write.mode('overwrite').partitionBy('year').parquet('/cartola/clean/times/')
#transforma arquivo em parquet
times_2017_parquet = spark.read.option('basePath', '/cartola/clean/times/').parquet('/cartola/clean/times/*')

In [28]:
times_2017_parquet.toPandas()

,ID,Nome,Abreviacao,Slug,year
0,373,Atlético-GO,ATL,Atlético - GO,2017
1,282,Atlético-MG,ATL,Atlético - MG,2017
2,293,Atlético-PR,ATL,Atlético - PR,2017
3,314,Avaí,AVA,Avaí - SC,2017
4,265,Bahia,BAH,Bahia - BA,2017
...,...,...,...,...,...
75,303,Ponte Preta,pon,ponte-preta,2015
76,314,Avaí,ava,avai,2015
77,315,Chapecoense,cha,chapecoense,2015
78,316,Figueirense,fig,figueirense,2015


## scouts_raw

In [63]:
# Analisando o arquivo 2014_lances.csv vimos que não é necessário processar esse arquivo, pois as informações relevantes estão em scouts_raw.

In [64]:
scouts_raw_2017_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2017/2017_scouts_raw.csv", header=True)

In [65]:
scouts_raw_2017_ano = scouts_raw_2017_df.withColumn('year', lit(2017))

In [66]:
scouts_raw_2017_ano.schema

StructType(List(StructField(_c0,StringType,true),StructField(A,StringType,true),StructField(CA,StringType,true),StructField(CV,StringType,true),StructField(DD,StringType,true),StructField(DP,StringType,true),StructField(FC,StringType,true),StructField(FD,StringType,true),StructField(FF,StringType,true),StructField(FS,StringType,true),StructField(FT,StringType,true),StructField(G,StringType,true),StructField(GC,StringType,true),StructField(GS,StringType,true),StructField(I,StringType,true),StructField(PE,StringType,true),StructField(PP,StringType,true),StructField(RB,StringType,true),StructField(SG,StringType,true),StructField(athletes.atletas.scout,StringType,true),StructField(atletas.apelido,StringType,true),StructField(atletas.atleta_id,StringType,true),StructField(atletas.clube.id.full.name,StringType,true),StructField(atletas.clube_id,StringType,true),StructField(atletas.foto,StringType,true),StructField(atletas.jogos_num,StringType,true),StructField(atletas.media_num,StringType,tr

In [71]:
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed('atletas.atleta_id', 'AtletaID')
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed('atletas.clube_id', 'ClubeID')
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed('atletas.pontos_num', 'Pontos')
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed('atletas.preco_num', 'Preco')
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed('atletas.variacao_num', 'PrecoVariacao')
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumnRenamed('atletas.preco_num', 'Preco')



In [68]:
scouts_raw_2017_ano = scouts_raw_2017_ano.drop('scout', 'atletas.apelido', 'atletas.clube.id.full.name', 'atletas.foto', 'atletas.jogos_num', 'atletas.nome', 'atletas.posicao_id', 'atletas.status_id',  )

In [72]:
scouts_raw_2017_ano = scouts_raw_2017_ano.withColumn("Pontos", scouts_raw_2017_ano["Pontos"].cast(FloatType()))

In [73]:
scouts_raw_2017_ano.write.mode('overwrite').partitionBy('year').parquet('/cartola/clean/scouts/')
#transforma arquivo em parquet
scouts_raw_2017_parquet = spark.read.option('basePath', '/cartola/clean/scouts/').parquet('/cartola/clean/scouts/*')

In [74]:
scouts_raw_2017_parquet.toPandas()

,AtletaID,Rodada,ClubeID,Participou,Posicao,Jogos,Pontos,PontosMedia,Preco,PrecoVariacao,...,RB,FC,GC,CA,CV,SG,DD,DP,GS,year
0,36540,0,FLA,None,None,None,0.0,None,5,0,...,None,None,None,None,None,None,None,None,None,2017
1,36612,0,PAL,None,None,None,0.0,None,8,0,...,None,None,None,None,None,None,None,None,None,2017
2,36943,0,ATL,None,None,None,0.0,None,10,0,...,None,None,None,None,None,None,None,None,None,2017
3,37245,0,BAH,None,None,None,0.0,None,4,0,...,None,None,None,None,None,None,None,None,None,2017
4,37246,0,SPO,None,None,None,0.0,None,4,0,...,None,None,None,None,None,None,None,None,None,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125648,97451,38,294,FALSE,None,None,0.0,0,1,0,...,0,0,0,0,0,0,0,0,0,2016
125649,97450,38,294,FALSE,None,None,0.0,0,1,0,...,0,0,0,0,0,0,0,0,0,2016
125650,97460,38,344,TRUE,None,None,2.0,2,1.78,0.78,...,1,1,0,0,0,0,0,0,0,2016
125651,82626,38,285,FALSE,None,None,0.0,0,1,0,...,0,0,0,0,0,0,0,0,0,2016


In [77]:
# -        Quantas partidas resultaram em empate?
pontos_por_atleta = scouts_raw_2017_ano[scouts_raw_2017_ano['year'] == 2017]

jogadores_2017 = jogadores_2017_parquet[jogadores_2017_parquet['year'] == 2017]

pontos_por_atleta = pontos_por_atleta.groupBy("AtletaID").agg(sum("Pontos").alias("SomaPontos"))

#count_result = count_result.withColumn('total_wins', count_result['count(1)'])

pontos_por_atleta = pontos_por_atleta.sort(pontos_por_atleta.SomaPontos.desc())

#count_result = count_result.drop('count(1)')

#count_result.show(5)


pontos_por_atleta.toPandas()

,AtletaID,SomaPontos
0,62121,280.199999
1,82455,221.000001
2,50284,203.299999
3,61149,196.299999
4,73281,182.500001
...,...,...
1010,97528,-6.500000
1011,98352,-6.800000
1012,90569,-7.200000
1013,37688,-8.100000


In [78]:
scouts_atletas = pontos_por_atleta.join(jogadores_2017, pontos_por_atleta.AtletaID == jogadores_2017.ID)
scouts_atletas = scouts_atletas.sort(scouts_atletas.SomaPontos.desc())
scouts_atletas.toPandas()

,AtletaID,SomaPontos,ID,Apelido,ClubeID,PosicaoID,year
0,62121,280.199999,62121,Vanderlei,277,1,2017
1,82455,221.000001,82455,Zé Rafael,265,4,2017
2,50284,203.299999,50284,Wilson,294,1,2017
3,61149,196.299999,61149,Lucca,303,5,2017
4,73281,182.500001,73281,Renê Júnior,265,4,2017
...,...,...,...,...,...,...,...
746,97528,-6.500000,97528,Frazan,266,3,2017
747,98352,-6.800000,98352,Bruno Guimarães,293,4,2017
748,90569,-7.200000,90569,Yuri,277,4,2017
749,37688,-8.100000,37688,Cícero,276,4,2017


<img src="https://s2.glbimg.com/WmFP3xwH6VxMuCvR72fl8ldWurA=/0x0:690x6668/984x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_bc8228b6673f488aa253bbcb03c80ec5/internal_photos/bs/2017/z/h/grGKnPRgmW6FtnMNiUNg/info-cartolao-2017-v4.jpg" width="800" height="400">

## DEMONSTRAÇÃO

In [ ]:
partidas_2014_df = spark.read.csv("/cartola/data/2014/2014_jogadores.csv", header=True)
partidas_2014_ct = partidas_2014_df.withColumn('time', regexp_replace('home_team', ' - RJ', ''))
final_partidas = partidas_2014_ct.withColumn('time_low', lower(col('time'))).show(truncate=False)

In [ ]:
with_ano_partidas = partidas_2014_ct.withColumn('ano', lit(2014)).show(truncate=False)
with_ano_partidas.show()

In [ ]:
Comando para sobrescrever arquivo caso já existente.
Agrupar scouts por ID e contar (caso tenha duplicidade)

In [85]:
inner_join = partidas_ids_2014_ano.join(times_2014_ano, partidas_ids_2014_ano.Casa == times_2014_ano.ID)
inner_join.toPandas()

,ID,Rodada,Casa,Visitante,PlacarCasa,PlacarVisitante,Resultado,ano,ID,Nome,Abreviacao,Slug,ano
0,179872,1,262,290,0,0,Empate,2014,262,flamengo,FLA,flamengo,2014
1,179873,1,266,316,3,0,Casa,2014,266,fluminense,FLU,fluminense,2014
2,179874,1,276,263,3,0,Casa,2014,276,são paulo,SAO,sao-paulo,2014
3,179875,1,277,292,1,1,Empate,2014,277,santos,SAN,santos,2014
4,179876,1,293,284,1,0,Casa,2014,293,atlético-pr,CAP,atletico-pr,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,180250,29,282,315,1,0,Casa,2014,282,atlético-mg,CAM,atletico-mg,2014
376,180251,29,287,283,0,1,Visitante,2014,287,vitória,VIT,vitoria,2014
377,180252,29,285,264,1,2,Visitante,2014,285,internacional,INT,internacional,2014
378,180253,29,316,294,4,0,Casa,2014,316,figueirense,FIG,figueirense,2014


In [ ]:
scouts_atletas = pontos_por_atleta.join(jogadores_2014_parquet, pontos_por_atleta.Atleta == jogadores_2014_parquet.ID)
scouts_atletas = scouts_atletas.sort(scouts_atletas.SomaPontos.desc())
scouts_atletas.toPandas()

In [ ]:
# -        Quantas partidas resultaram em empate?
pontos_por_atleta = scouts_raw_2014_ano.groupBy("Atleta").agg(sum("Pontos").alias("SomaPontos"))

#count_result = count_result.withColumn('total_wins', count_result['count(1)'])

pontos_por_atleta = pontos_por_atleta.sort(pontos_por_atleta.SomaPontos.desc())

#count_result = count_result.drop('count(1)')

#count_result.show(5)


pontos_por_atleta.toPandas()